In [2]:
import math
import itertools
import pandas

In [3]:
pandas.set_option('display.max_rows', None)

In [4]:
enum = { "nada": 0, "básico": 1, "médio": 2, "avançado": 3 }
inverse_enum = { 0: "nada", 1: "básico", 2: "médio", 3: "avançado" }
r = itertools.combinations_with_replacement([
    enum["nada"],
    enum["básico"],
    enum["médio"],
    enum["avançado"]
], 3)
r = list(r)

In [6]:
x = [enum["nada"],
    enum["básico"],
    enum["médio"],
    enum["avançado"]]


r = [p for p in itertools.product(x, repeat=3)]

r = list(r)

In [8]:
def format_result(i, sim):
    return {
        "Excel": inverse_enum[i[0]],
        "PPT": inverse_enum[i[1]],
        "Word": inverse_enum[i[2]],
        "Similaridade": sim
    }

## Problema com as medidas antigas

Definir o denominador por um valor variável causa problemas entre as medidas pois é como se não estivéssemos medindo a similaridade com expectativas iguais, portanto, as novas medidas incluem um valor fixo no numerador para todos os objetos e cenários

## Sem match | Pesos iguais

$$S_{i} =
  \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik}}{|L| - 1} + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}                             {\sum_{k = 0}^{N-1} \frac{|L| - 1}{|L| - 1} + c_{k,max} \cdot (c_{k,max})^{(|L| - 1) - (|L| - 1)}}  = \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik}}{|L| - 1} + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}
      {\sum_{k = 0}^{N-1} 1 + 1 \cdot (1 + 1)^0} = 
 \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik}}{|L| - 1} + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}
                {\sum_{k = 0}^{N-1} 2}$$

$$S_{i} = \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik}}{|L| - 1} + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}
                {2N - 2}$$

$$ 0 \leq c_k \leq 1 $$
$$ c_{k,min} = 0; c_{k,max} = 1 $$

In [353]:
def no_match_equal_weights(i, C = [1,1,1]):
    numerator = 0
    denominator = 0
    
    for _i,  _c in list(zip(i, C)):
        denominator += 2
        numerator += (_i/3) + _c*(_c+1)**(_i - 3)
        
    return format_result(i, numerator/denominator)

## Sem match | Pesos diferentes

$$S_{ij} =
  \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik}}{|L| - 1} \cdot (\frac{r_{jk}}{|L| - 1} + 1) + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}
        {\sum_{k = 0}^{N-1} \frac{|L| - 1}{|L| - 1} \cdot (\frac{|L| - 1}{|L| - 1} + 1)  + c_{k,max} \cdot (c_{k,max})^{(|L| - 1) - (|L| - 1)}} =
\dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik}}{|L| - 1} \cdot (\frac{r_{jk}}{|L| - 1} + 1) + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}
      {\sum_{k = 0}^{N-1} 2 + 1 \cdot (1 + 1)^0} = 
 \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik}}{|L| - 1} \cdot (\frac{r_{jk}}{|L| - 1} + 1) + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}
                {\sum_{k = 0}^{N-1} 3}$$

$$S_{ij} = \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik}}{|L| - 1} \cdot (\frac{r_{jk}}{|L| - 1} + 1) + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}
                {3N - 3}$$

$$ 0 \leq c_k \leq 1 $$
$$ c_{k,min} = 0; c_{k,max} = 1 $$

In [352]:
def no_match_diff_weights(i, j = [3, 3, 3], C = [1,1, 1]):
    numerator = 0
    denominator = 0
    
    for _i, _j, _c in list(zip(i, j, C)):
        denominator += 3
        numerator += (_i/3)*((_j/3)+1) + _c*(_c+1)**(_i-3)
            
    return format_result(i, numerator/denominator)

## Com match | Pesos diferentes

$$S_{ij} =
  \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik} - r_{jk}}{|L| - 1} \cdot (\frac{r_{jk}}{|L| - 1} + 1) + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}
        {\sum_{k = 0}^{N-1} (\frac{r_{jk}}{|L| - 1} + 1) + d_{ijk}} =
  \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik} - r_{jk}}{|L| - 1} \cdot (\frac{r_{jk}}{|L| - 1} + 1) + c_{k} \cdot (c_{k})^{r_{ik} - (|L| - 1)}}
        {\sum_{k = 0}^{N-1} (\frac{r_{jk}}{|L| - 1} + 1) + d_{ijk}}$$

$$ d_{ijk} = \begin{cases} 1, & \mbox{if } r_{ik} > r_{jk} \\ 0, & \mbox{if } r_{ik} \leq r_{jk} \end{cases} $$

$$ 0 \leq c_k \leq 1 $$
$$ c_{k,min} = 0; c_{k,max} = 1 $$

In [10]:
def with_match_diff_weights(i, j = [3, 3, 3], C = [1,1,1]):
    numerator = 0
    denominator = 0
    
    for _i, _j, _c in list(zip(i, j, C)):        
        numerator += ((_i-_j)/3)*((_j/3)+1) + _c*(_c+1)**(_i - 3)
        
        if _i > _j:
            denominator += ((_j/3) + 1) + 1        
        else:
            denominator += ((_j/3) + 1)
        
    return format_result(i, numerator/denominator)

# Generalizações


### Sem match | Pesos iguais

1. **Para qualquer valor de $ i_k $, $i_k + 1$ sempre terá um valor de similaridade maior que $i_k$**;
   - ```py
    >>> no_match_equal_weights([0, 1], [1, 1]), no_match_equal_weights([0, 2], [1, 1])
    ({'Similaridade': 0.17708333333333331}, {'Similaridade': 0.32291666666666663})
    >>> no_match_equal_weights([2, 1], [1, 1]), no_match_equal_weights([2, 3], [1, 1])
    ({'Similaridade': 0.43749999999999994}, {'Similaridade': 0.7916666666666666})
    ```

2. **Dado um objeto de $i_k$ com $C + n$, o seu valor de similaridade será maior do que com $C$;** Depende dos outros valores de $i$
   - ```py
    >>> no_match_equal_weights([3, 0], [1, 0]), no_match_equal_weights([3, 0], [1, 1])
    ({'Similaridade': 0.5}, {'Similaridade': 0.53125})
    >>> no_match_equal_weights([3, 1], [1, 0]), no_match_equal_weights([3, 1], [1, 1])
    ({'Similaridade': 0.5833333333333334}, {'Similaridade': 0.6458333333333333})
    >>> no_match_equal_weights([2, 1], [1, 0]), no_match_equal_weights([2, 1], [1, 1])
    ({'Similaridade': 0.37499999999999994}, {'Similaridade': 0.43749999999999994})
    >>> no_match_equal_weights([0, 1], [1, 0]), no_match_equal_weights([0, 1], [1, 1])
    ({'Similaridade': 0.11458333333333333}, {'Similaridade': 0.17708333333333331})
    ```

3. **Se $i_k = |L| - 1$ e $C = 1$, $S_i = 1$**
    - ```py
    >>> no_match_equal_weights([3], [0])
    {'Similaridade': 1.0}
    ```
    
4. **Se $i_k = |L| - 1$ e $C < 1$, $S_i < 1$**
    - ```py
    >>> no_match_equal_weights([3], [0.5])
    {'Similaridade': 0.75}
    ```
     
3. **Se $i_k = 0$ e $C = 0$, $S_i = 0$**
    - ```py
    >>> no_match_equal_weights([0], [0])
    {'Similaridade': 1.0}
    ```
    
4. **Se $i_k = 0$ e $C > 0$, $S_i > 0$**
    - ```py
    >>> no_match_equal_weights([0], [0.5])
    {'Similaridade': 0.07407407407407407}
    ```
     
4. **$S_i \in [0, 1]$**
    - ```py
    >>> no_match_equal_weights([0], [0]), no_match_equal_weights([3], [1])
    ({'Similaridade': 0.0}, {'Similaridade': 1.0})
    ```

### Sem match | Pesos diferentes

1. **Para um mesmo valor de $ i_k $ entre objetos, um objeto com $ j_k + 1 $ resultará em um valor de similaridade maior de que um objeto com $ j_k $.**

  - ```py
  >>> no_match_diff_weights([2, 1], [2, 1], [1, 1]), no_match_diff_weights([2, 1], [2, 2], [1, 1])
  ({'Similaridade': 0.38425925925925924}, {'Similaridade': 0.40277777777777773})
  >>> no_match_diff_weights([0, 1], [2, 1]), no_match_diff_weights([0, 1], [2, 2])
  ({'Similaridade': 0.13657407407407407}, {'Similaridade': 0.1550925925925926})
  ```

2. **Para um mesmo valor de $j_k$ entre objetos, um objeto com $i_k + 1$ terá um valor de similaridade maior do que um objeto com $j_k$**
  - ```py
  >>> no_match_diff_weights([2, 0], [2, 2], [1, 1]), no_match_diff_weights([2, 1], [2, 2], [1, 1])
  ({'Similaridade': 0.2893518518518518}, {'Similaridade': 0.40277777777777773})
  >>> no_match_diff_weights([1, 2], [3, 1], [1, 1]), no_match_diff_weights([1, 3], [3, 1], [1, 1])
  ({'Similaridade': 0.38425925925925924}, {'Similaridade': 0.5416666666666666})
  ```

3. **Para um objeto com $i_k$, $j_k$ e $C$ e outro com $i_k$, $j_k$ e $C + 1$, o primeiro resultará um valor de similaridade menor que o segundo;**

  - ```py
  >>> no_match_diff_weights([2, 1], [1, 3], [1, 0]), no_match_diff_weights([2, 1], [1, 3], [1, 1])
  ({'Similaridade': 0.34259259259259256}, {'Similaridade': 0.38425925925925924})
  >>> no_match_diff_weights([3, 1], [1, 3], [1, 0]), no_match_diff_weights([3, 1], [1, 3], [1, 1])
  ({'Similaridade': 0.49999999999999994}, {'Similaridade': 0.5416666666666666})
  >>> no_match_diff_weights([0, 1], [1, 3], [1, 0]), no_match_diff_weights([0, 1], [1, 3], [1, 1])
  ({'Similaridade': 0.13194444444444445}, {'Similaridade': 0.17361111111111108})
  >>> no_match_diff_weights([3, 1], [1, 3], [1, 0]), no_match_diff_weights([3, 1], [1, 3], [1, 1])
  ({'Similaridade': 0.49999999999999994}, {'Similaridade': 0.5416666666666666})
  ```

6. $S_{ij} \in [0, 1]$
  - ```py
  >>> no_match_diff_weights([3, 3], [3, 3], [1, 1]), no_match_diff_weights([0, 0], [0, 0], [0, 0])
  ({'Similaridade': 1.0}, {'Similaridade': 0.0})
  ```

### Com match | Pesos diferentes

1. **Para um mesmo valor de $ i_k $ entre objetos, um objeto com $ j_k + 1 $ resultará em um valor de similaridade menos de que um objeto com $ j_k $.**

  - ```py
  >>> with_match_diff_weights([2, 1], [2, 1], [0, 0]), with_match_diff_weights([2, 1], [2, 2], [0, 0])
  ({'Similaridade': 0.0}, {'Similaridade': -0.16666666666666666})
  >>> with_match_diff_weights([0, 1], [2, 1], [1, 1]), with_match_diff_weights([0, 1], [2, 2], [1, 1])
  ({'Similaridade': -0.24537037037037032}, {'Similaridade': -0.3875})
  ```

2. **Para um mesmo valor de $j_k$ entre objetos, um objeto com $i_k + 1$ terá um valor de similaridade maior do que um objeto com $j_k$**
  - ```py
  >>> with_match_diff_weights([2, 0], [2, 2], [0, 0]), with_match_diff_weights([2, 1], [2, 2], [0, 0])
 ({'Similaridade': -0.3333333333333333}, {'Similaridade': -0.16666666666666666})
  >>> with_match_diff_weights([1, 2], [3, 1], [1, 1]), with_match_diff_weights([1, 3], [3, 1], [1, 1])
  ({'Similaridade': -0.03205128205128204}, {'Similaridade': 0.1858974358974359})
  ```

3. **Para um objeto com $i_k$, $j_k$ e $C$ e outro com $i_k$, $j_k$ e $C + 1$, o primeiro terá um valor de similaridade menor que o segundo;**

  - ```py
  >>> with_match_diff_weights([2, 1], [1, 3], [1, 0]), with_match_diff_weights([2, 1], [1, 3], [1, 1])
  ({'Similaridade': -0.08974358974358974}, {'Similaridade': -0.03205128205128204})
  >>> with_match_diff_weights([3, 1], [1, 3], [1, 0]), with_match_diff_weights([3, 1], [1, 3], [1, 1])
  ({'Similaridade': 0.12820512820512822}, {'Similaridade': 0.1858974358974359})
  >>> with_match_diff_weights([0, 0, 1], [2, 3, 0], [1, 1, 0]), with_match_diff_weights([0, 0, 1], [2, 3, 0], [1, 1, 1])
  ({'Similaridade': -0.446078431372549}, {'Similaridade': -0.4019607843137255})
  ```

5. **Dado que $i_k$ de um objeto é menor que $j_k$, o valor de similaridade será no intervalo $[-1, 0[$**
  - ```py
  >>> with_match_diff_weights([0], [3], [0]), with_match_diff_weights([2], [3], [1])
  ({'Similaridade': -1.0}, {'Similaridade': -0.08333333333333331})
  ```

6. **Dado que $i_k$ de um objeto é maior que $j_k$, o valor de similaridade será no intervalo $]0, 1]$**
  - ```py
  >>> with_match_diff_weights([1], [0], [0]), with_match_diff_weights([3], [0], [1])
  ({'Similaridade': 0.16666666666666666}, {'Similaridade': 1.0})
  ```
7. $S_{ij} \in[-1,1]$, porém ocupará um subintervalo de módulo igual à $1$, por exemplo, de $-0.55$ a $0.45$ e $-0.9$ a $0.1$

-----------------------


## Mérito: C

```py
>>> [no_match_equal_weights([0, 0], [1, 1]),
 no_match_equal_weights([1, 0], [1, 1]),
 no_match_equal_weights([2, 0], [1, 1]),
 no_match_equal_weights([3, 0], [1, 1])]
[{'Similaridade': 0.0625},
 {'Similaridade': 0.17708333333333331},
 {'Similaridade': 0.32291666666666663},
 {'Similaridade': 0.53125}]
 
>>> [no_match_equal_weights([0, 0], [1, 0]),
 no_match_equal_weights([1, 0], [1, 0]),
 no_match_equal_weights([2, 0], [1, 0]),
 no_match_equal_weights([3, 0], [1, 0])]
[{'Similaridade': 0.03125},
 {'Similaridade': 0.14583333333333331},
 {'Similaridade': 0.29166666666666663},
 {'Similaridade': 0.5}]


>>> [with_match_diff_weights([0, 2, 3], [2,2,2], [0, 0, 1]),
with_match_diff_weights([3, 2, 0], [2,2,2], [0, 0, 1])]
[{'Similaridade': 0.07407407407407407}, {'Similaridade': -0.07175925925925924}]
```

# Tests

In [16]:
result = list(map(lambda i: with_match_diff_weights(i, [1, 2, 3], [0, 0, 0]), list(r)))

for i, v in enumerate(result):
    result[i]['idx'] = i
    
#result = sorted(result, key=lambda x: x["Similaridade"])
df = pandas.DataFrame(result)
df

,Excel,PPT,Word,Similaridade,idx
0,nada,nada,nada,-0.711111,0
1,nada,nada,básico,-0.577778,1
2,nada,nada,médio,-0.444444,2
3,nada,nada,avançado,-0.311111,3
4,nada,básico,nada,-0.600000,4
5,nada,básico,básico,-0.466667,5
6,nada,básico,médio,-0.333333,6
7,nada,básico,avançado,-0.200000,7
8,nada,médio,nada,-0.488889,8
9,nada,médio,básico,-0.355556,9
